In [1]:
from utils.df_handle import *

Default bq project:  spatial-vision-343005.biteam


In [ ]:
# from utils.df_handle import *
import pendulum
from airflow import DAG
from airflow.operators.dummy_operator import DummyOperator
from airflow.operators.python_operator import PythonOperator

local_tz = pendulum.timezone("Asia/Bangkok")

name='OC'
prefix='SYNC_'
csv_path = '/usr/local/airflow/plugins'+'/'

dag_params = {
    'owner': 'airflow',
    "depends_on_past": False,
    'start_date': datetime(2022, 5, 10, tzinfo=local_tz),
    'email_on_failure': True,
    'email_on_retry': False,
    'email':['duyvq@merapgroup.com', 'vanquangduy10@gmail.com'],
    'do_xcom_push': False,
    'execution_timeout':timedelta(seconds=300)
    # 'retries': 3,
    # 'retry_delay': timedelta(minutes=10),
}

dag = DAG(prefix+name,
          catchup=False,
          default_args=dag_params,
          schedule_interval= '0 1 * * *',
          tags=[prefix+name, 'Sync', '30mins']
)

In [4]:
start_date = '2022-01-01'
datenow = datetime.now().strftime("%Y-%m-%d")
datenow_mns1 = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")
fdom = datetime.now().replace(day=1).strftime("%Y-%m-%d")
datenow_mns45 = (datetime.now() - timedelta(days=45)).strftime("%Y-%m-%d")

In [5]:
from_tb = "PPC_OutsideChecking"

In [6]:
table_name = "sync_dms_oc"
table_temp = "sync_dms_oc_temp"

In [9]:
sql = \
f"""
DECLARE @from DATE = '{start_date}'
DECLARE @cc VARCHAR(100)= (SELECT TextVal FROM vs_SYS_Configurations WHERE Code='ImagePublic')
DECLARE @to DATE = '2022-01-31'
select
SlsperID,
CustID,
VisitDate,
NoteID,
BranchID,
oc.Note,
rs.Descr,
SalesID,
Distance,
CheckInType = CASE WHEN oc.NoteID = 2 THEN N'Giao Hàng' WHEN oc.NoteID = 1 THEN N'Thu Nợ' ELSE N'Bán Hàng' end,
ImageFileName = CASE WHEN oc.ImageFileName ='' THEN '' ELSE @cc+oc.ImageFileName END
from PPC_OutsideChecking oc
LEFT JOIN dbo.OM_ReasonCodePPC rs WITH (NOLOCK) ON oc.ReasonCode=rs.Code AND rs.Type IN ('DELIDISTANCE','DS','REDEBTDISTANCE')
where oc.VisitDate >= @from
"""

In [10]:
print(sql)


DECLARE @from DATE = '2022-01-01'
DECLARE @cc VARCHAR(100)= (SELECT TextVal FROM vs_SYS_Configurations WHERE Code='ImagePublic')
DECLARE @to DATE = '2022-01-31'
select
SlsperID,
CustID,
VisitDate,
NoteID,
BranchID,
oc.Note,
rs.Descr,
SalesID,
Distance,
CheckInType = CASE WHEN oc.NoteID = 2 THEN N'Giao Hàng' WHEN oc.NoteID = 1 THEN N'Thu Nợ' ELSE N'Bán Hàng' end,
ImageFileName = CASE WHEN oc.ImageFileName ='' THEN '' ELSE @cc+oc.ImageFileName END
from PPC_OutsideChecking oc
LEFT JOIN dbo.OM_ReasonCodePPC rs WITH (NOLOCK) ON oc.ReasonCode=rs.Code AND rs.Type IN ('DELIDISTANCE','DS','REDEBTDISTANCE')
where oc.VisitDate >= @from



In [11]:
df = get_ms_df(sql)
df['inserted_at'] = datetime.now()
bq_values_insert(df, f"{table_temp}", 3)
sql = \
f"""
DROP TABLE IF EXISTS biteam.{table_name};
CREATE TABLE biteam.{table_name} LIKE biteam.{table_temp}
PARTITION BY DATE(visitdate)
CLUSTER BY branchid,salesid,custid,slsperid
"""
execute_bq_query(sql)

1it [00:17, 17.86s/it]


In [4]:
dsql = \
f"""
delete from biteam.{table_name} where date(visitdate) >= '{datenow_mns45}'
"""
print("delete_sql: ", dsql)
execute_bq_query(dsql)

In [12]:
# df = get_ms_df(sql)
bq_values_insert(df, f"{table_name}", 2)

1it [00:28, 28.36s/it]


In [ ]:
# dummy_start = DummyOperator(task_id="dummy_start", dag=dag)

# insert = PythonOperator(task_id="insert", python_callable=insert, dag=dag)

# update = PythonOperator(task_id="update", python_callable=update, dag=dag)

# dummy_start >> insert >> update

In [8]:
sql = \
"""
DECLARE @from DATE = '2022-03-01'
DECLARE @cc VARCHAR(100)= (SELECT TextVal FROM vs_SYS_Configurations WHERE Code='ImagePublic')
DECLARE @to DATE = '2022-05-31'
select
SlsperID,
CustID,
VisitDate,
NoteID,
BranchID,
oc.Note,
rs.Descr,
SalesID,
Distance,
CheckInType = CASE WHEN oc.NoteID = 2 THEN N'Giao Hàng' WHEN oc.NoteID = 1 THEN N'Thu Nợ' ELSE N'Bán Hàng' end,
ImageFileName = CASE WHEN oc.ImageFileName ='' THEN '' ELSE @cc+oc.ImageFileName END
from PPC_OutsideChecking oc
LEFT JOIN dbo.OM_ReasonCodePPC rs WITH (NOLOCK) ON oc.ReasonCode=rs.Code AND rs.Type IN ('DELIDISTANCE','DS','REDEBTDISTANCE')
where cast(oc.VisitDate as DATE) >= @from and cast(oc.VisitDate as DATE) <= @to
"""

In [9]:
df = get_ms_df(sql)

In [10]:
df.shape

(167864, 11)

In [13]:
table_name = "sync_dms_oc"

In [14]:
bq_values_insert(df, f"{table_name}", 2)

1it [00:15, 15.49s/it]


In [11]:
df['inserted_at'] = datetime(2022,5,31)

In [12]:
df.head()

,SlsperID,CustID,VisitDate,NoteID,BranchID,Note,Descr,SalesID,Distance,CheckInType,ImageFileName,inserted_at
0,MR0025,000006,2022-03-19 07:59:10,2,MR0001,,KHÁC (Nhập rõ lý do),36ca4740-b0d1-4b97-907b-37460a73afb9,0,Giao Hàng,,2022-05-31
1,MR0025,000317,2022-03-08 09:54:26,2,MR0001,,KHÁC (Nhập rõ lý do),86294e9e-82ff-46e5-835b-0843a4827db4,4266,Giao Hàng,,2022-05-31
2,MR0025,000317,2022-04-08 15:43:17,2,MR0001,,KHÁC (Nhập rõ lý do),d6b4bc6d-a76f-4790-b843-197ec1a7878a,3371,Giao Hàng,https://dms.phanam.com.vn/IMG/MR0001/MR0025/20...,2022-05-31
3,MR0025,000318,2022-03-08 09:52:49,2,MR0001,,KHÁC (Nhập rõ lý do),f317de23-b3da-4fee-9482-841b3de39e37,0,Giao Hàng,,2022-05-31
4,MR0025,000318,2022-03-08 09:53:42,2,MR0001,,KHÁC (Nhập rõ lý do),36c835b6-8aca-4d84-a465-52f866950c5f,0,Giao Hàng,,2022-05-31
